# prompt_builder

> A nano-library for filling the fields of a prompt template, partially and in any order.

In [6]:
#| default_exp core

In [7]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export

from typing import Dict, Set
from string import Formatter


def fields_needed(format_string:str) -> Set[str]:
    "Returns fields needed to complete format_string"
    return set(fname for (_,fname,_,_) in Formatter().parse(format_string) if fname)


def substitute_vals(format_string:str,**vals) -> str:
    "Substitutes fields from vals into format_string"
    still_needed = set(fields_needed(format_string)) - set(vals.keys())
    missing_dict = {k:( '{' + k + '}' ) for k in still_needed}
    return format_string.format(**(vals | missing_dict))


class Prompt:
    def __init__(self,format_string):
      'Initializees a prompt from a format string'
      self.format_string = format_string
      self.values:Dict[str,str] = {}
    def add(self,**new_values):
      "Fill the prompt's fields with new values"
      self.values = self.values | new_values
      return self
    def needed(self) -> Set[str]:
      "Returns the fields not yet filled"
      return set(fields_needed(self.format_string)) - set(self.values.keys())
    def text(self) -> str:
      "Returns the prompt, filled completely or partially"
      return substitute_vals(self.format_string,**self.values)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()